<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#housekeeping" data-toc-modified-id="housekeeping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>housekeeping</a></span><ul class="toc-item"><li><span><a href="#languages-&amp;-packages" data-toc-modified-id="languages-&amp;-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>languages &amp; packages</a></span></li><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>env set-up</a></span></li><li><span><a href="#parameters" data-toc-modified-id="parameters-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>parameters</a></span></li><li><span><a href="#equation-system" data-toc-modified-id="equation-system-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>equation system</a></span></li><li><span><a href="#equation-system-visualization" data-toc-modified-id="equation-system-visualization-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>equation system visualization</a></span></li></ul></li><li><span><a href="#numeric-solve" data-toc-modified-id="numeric-solve-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>numeric solve</a></span><ul class="toc-item"><li><span><a href="#whole-ODE-system" data-toc-modified-id="whole-ODE-system-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>whole ODE system</a></span></li><li><span><a href="#$P$-&amp;-partial-$C$-equation-isolation" data-toc-modified-id="$P$-&amp;-partial-$C$-equation-isolation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>$P$ &amp; partial-$C$ equation isolation</a></span></li><li><span><a href="#$B$-&amp;-partial-$C$-equation-isolation" data-toc-modified-id="$B$-&amp;-partial-$C$-equation-isolation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>$B$ &amp; partial-$C$ equation isolation</a></span></li></ul></li><li><span><a href="#conclusion" data-toc-modified-id="conclusion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>conclusion</a></span></li><li><span><a href="#ODE-system-variation" data-toc-modified-id="ODE-system-variation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ODE system variation</a></span><ul class="toc-item"><li><span><a href="#parameters" data-toc-modified-id="parameters-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>parameters</a></span></li><li><span><a href="#equation-system" data-toc-modified-id="equation-system-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>equation system</a></span></li><li><span><a href="#equation-system-visualization" data-toc-modified-id="equation-system-visualization-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>equation system visualization</a></span></li></ul></li></ul></div>

# housekeeping
This notebook is for analytically (2nd attempt) solving parameters in the set of ODEs for eco-bioelectric cell (EBC).

## languages & packages
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

## env set-up

In [1]:
using PyCall, DataFrames, SymPy
sympy.init_printing() ## use other py3 function not implemented in native Julia-lang, http://mth229.github.io/symbolic.html
println()

In [2]:
C,P,B,K_P, g_P,e_P,R_P,m_P, g_B,e_B,R_B,m_B = symbols("C P B K_P g_P e_P R_P m_P g_B e_B R_B m_B", positive=true)

(C, P, B, K_P, g_P, e_P, R_P, m_P, g_B, e_B, R_B, m_B)

## parameters
symbol | unit | meaning
--- | --- | ---
$C$ | kgCm$^-3$ | system organic __C__arbon pool concentration
$P$ | kgCm$^-3$ | system __P__hytoplankton Carbon pool concentration
$B$ | kgCm$^-3$ | system _E. coli_ (__B__acteria) Carbon pool concentration
$K_P$ | kgCm$^-3$ | system phytoplankton Carbon pool carrying capacity
$g_P$ | t$^{-1}$ | growth rate of $P$
$e_P$ | no | carbon assimilation efficiency of $P$
$R_P$ | t$^{-1}$ | respiration rate of $P$
$m_P$ | t$^{-1}$ | death rate of $P$
$g_B$ | m$^3$(kgC$\cdot$t)$^{-1}$ | growth rate per $B$ pool concentration unit
$e_B$ | no | absorption efficiency of $B$
$R_B$ | t$^{-1}$ | respiration rate of $B$
$m_B$ | t$^{-1}$ | death rate of $B$

## equation system
\begin{equation}
\left\{\begin{array}{rl}
C'(t) &= (g_P(1-\varepsilon_P)\left(1-\frac{P}{K_P}\right) + m_P)P + (g_B(1-\varepsilon_B)C + m_B)B\\
P'(t) &= (g_P\varepsilon_P \left(1-\frac{P}{K_P}\right) - R_P - m_P)P\\
B'(t) &= (g_B\varepsilon_BC - R_B - m_B)B\\
\end{array}\right.
\end{equation}

## equation system visualization
![](graph/ebc5.png)

# numeric solve

In [3]:
C_eqm = Eq((g_P*(1 -e_P)*(1 -P/K_P) +m_P)*P + (g_B*(1 -e_B)*C +m_B)*B, 0)
P_eqm = Eq((g_P*e_P*(1 -P/K_P) -R_P -m_P)*P, 0)
B_eqm = Eq((g_B*e_B*C -R_B -m_B)*B, 0)

B⋅(C⋅e_B⋅g_B - R_B - m_B) = 0

## whole ODE system

In [4]:
sol = solve([C_eqm, P_eqm, B_eqm], [C, P, B])
DataFrame(
    var = ["C","P","B"],
    value = [sol[1][1],sol[1][2],sol[1][3]]
)

,var,value
,String,Sym
1,C,(R_B + m_B)/(e_B*g_B)
2,P,-K_P*(R_P - e_P*g_P + m_P)/(e_P*g_P)
3,B,K_P*e_B*(R_P - e_P*g_P + m_P)*(R_P*e_P - R_P - m_P)/(e_P^2*g_P*(R_B*e_B - R_B - m_B))


In [5]:
sol[1][1] # C

R_B + m_B
─────────
 e_B⋅g_B 

So $g_B$, $e_B$, $R_B$, $m_B$ can be any values within predefined ranges

In [6]:
P0 = Eq(sol[1][2],0) # P

-K_P⋅(R_P - e_P⋅g_P + m_P)     
─────────────────────────── = 0
          e_P⋅g_P              

In [7]:
solve(P0, [g_P,e_P,R_P,m_P,K_P])

1-element Array{NTuple{5,Sym},1}:
 ((R_P + m_P)/e_P, e_P, R_P, m_P, K_P)

[default sol'n]: $e_P$, $R_P$, $m_P$, $K_P$ can be any value; $g_P$ can only be larger than ($\dfrac{R_P+m_P}{e_P}$)  
Or other equivalent alternatives  
__NOTE__: a fast-growing, high-efficiency phytoplankton is preferred & advantageous

In [8]:
B0 = Eq(sol[1][3],0) # B

K_P⋅e_B⋅(R_P - e_P⋅g_P + m_P)⋅(R_P⋅e_P - R_P - m_P)    
─────────────────────────────────────────────────── = 0
              2                                        
           e_P ⋅g_P⋅(R_B⋅e_B - R_B - m_B)              

In [9]:
solve(B0, [C,P, g_P,e_P,R_P,m_P, g_B,e_B,R_B,m_B])

2-element Array{NTuple{10,Sym},1}:
 (C, P, g_P, e_P, m_P/(e_P - 1), m_P, g_B, e_B, R_B, m_B)
 (C, P, g_P, e_P, e_P*g_P - m_P, m_P, g_B, e_B, R_B, m_B)

Two eqm positions exist for $B$ > 0  
All but one parameters can take any values  
$R_P$ can only be either ($\dfrac{m_P}{e_P-1}$) or ($e_P*g_P-m_P$); however ($\dfrac{m_P}{e_P-1}$) < 0 $\because$ 0 $\leq e_P\leq$ 1, so the only solution is ($e_P*g_P-m_P$) with ($e_P*g_P$) > $m_P$ (the former solution has to be rejected as $R_P$ is a non-negative number)  
__NOTE__: the above deduction also favours high growth rate high efficiency low death rate phytoplankton lineages

## $P$ & partial-$C$ equation isolation

In [10]:
C_eqmP = Eq((g_P*(1 -e_P)*(1 -P/K_P) +m_P)*P, 0)
sol = solve([C_eqmP, P_eqm], [C, P])

0-element Array{Any,1}

## $B$ & partial-$C$ equation isolation

In [11]:
C_eqmB = Eq((g_B*(1 -e_B)*C +m_B)*B, 0)
sol = solve([C_eqmB, B_eqm], [C, B])

0-element Array{Any,1}

# conclusion
From above numeric analysis, the system only limits the use of phytoplankton lineages.  High growth rate, high carbon-assimilation efficiency (i.e. low leakage) and low death rate lineages are required.  
On the other hand, flexible _E. coli_ limits are observed.  Hence the idea of incorporating high CUE phytoplankton, low CUE _E. coli_ works, as long as electricity generation calculation also allows low CUE _E. coli_ to serve.

# ODE system variation
## parameters
symbol | unit | meaning
--- | --- | ---
$C$ | kgCm$^-3$ | system organic __C__arbon pool concentration
$P$ | kgCm$^-3$ | system __P__hytoplankton Carbon pool concentration
$B$ | kgCm$^-3$ | system _E. coli_ (__B__acteria) Carbon pool concentration
$a_P$ | $m^3(kgC)^{-1}$ | intraspecific density dependence coefficient
$g_P$ | t$^{-1}$ | growth rate of $P$
$e_P$ | no | carbon assimilation efficiency of $P$
$R_P$ | t$^{-1}$ | respiration rate of $P$
$m_P$ | t$^{-1}$ | death rate of $P$
$g_B$ | m$^3$(kgC$\cdot$t)$^{-1}$ | growth rate per $B$ pool concentration unit
$e_B$ | no | absorption efficiency of $B$
$R_B$ | t$^{-1}$ | respiration rate of $B$
$m_B$ | t$^{-1}$ | death rate of $B$

## equation system
\begin{equation}
\left\{\begin{array}{rl}
C'(t) &= (g_P(1-\varepsilon_P) + a_PP + m_P)P + (g_B(1-\varepsilon_B)C + m_B)B\\
P'(t) &= (g_P\varepsilon_P - a_PP - R_P - m_P)P\\
B'(t) &= (g_B\varepsilon_BC - R_B - m_B)B\\
\end{array}\right.
\end{equation}

## equation system visualization
![](graph/ebc6.png)